# CSV 4 - número de repetições do termo "distanciamento social" por dia

### Importações necessárias

In [148]:
#para a função json_to_dataframe
import nltk
import pandas as pd
from IPython.display import clear_output

#para a função contagem_bigramas_data
from collections import defaultdict

#para a função tokenizar_texto
from nltk.corpus import stopwords #-----> talvez seja necessário baixar as stopwords com o comando no CMD: python -m nltk.downloader stopwords
from string import punctuation
from nltk.tokenize import TweetTokenizer
import re


### Função que lê um json e retorna um dataframe reduzido
##### OBS: o arquivo json é lido por pedaços, que são conjuntos delimitados de linhas (devem ser passados no parâmetro "chunk")

In [149]:
### É preciso passar como parâmetro o nome do arquivo Json, o tamanho do chunk e uma lista com o nome das colunas que se quer obter
def json_to_dataframe(nome_json, colunas, chunk):
    
    contador = 0
    dados = pd.read_json(nome_json, convert_dates= False, lines=True, chunksize=chunk)
    pd.set_option("display.max_colwidth", 100000) #Não deixa trazer visualizações de dataframe truncadas (não interfere no resultado)
    
    
    for df_tweets_retweets in dados: ##->> Acessa os dados dos tweets e retweets de um chunk, gerando um dataframe do tamanho do chunk.                         
        
        #clear_output()
        contador += chunk
        print("%s tweets processados..." %(str(contador))) ## Mostra o número de tweets processados a cada chunk que passa

        
        yield df_tweets_retweets[colunas] ##cria um generator, que continua retornando dataframes enquanto o loop existir
    

### Função que conta ocorrências de bigramas por data
##### obs: Precisa receber uma lista de bigramas e um generator de dataframe que contenha as colunas "text" e "created_at"

In [150]:
def contagem_bigramas_data(bigrama_lst, generator_df):   #bigrama precisa ser uma string contendo duas palavras separadas por espaço ex: 'distanciamento social'
    
    dfldict_cont_bigramas = defaultdict(int)        ##Criação de um tipo diferente de dicionário...
    defaultdict_datas_bigramas = defaultdict(dict)               


    for dataframe in generator_df: #'Percorre' o generator, extraindo os dataframes 
        
        for linha in dataframe.values: #percorre as linhas do dataframe (as operações principais vão aqui)
            
            bigramas_tweet = list(nltk.bigrams(tokenizar_texto(linha[0]))) #tratamento de textos                                
            
            for bigrama in bigrama_lst:
                
                bigrama_tlp = tuple(bigrama.lower().split()) #transforma o bigrama em tupla
                
                if bigrama_tlp in bigramas_tweet: ##imprime só os tweets que contenham o bigrama ('distanciamento', 'social')
                    
                    dfldict_cont_bigramas[bigrama_tlp] += 1
                    data = linha[1]['$date'][:10]                             #essa linha deixa a data e tira a hora
                    defaultdict_datas_bigramas[data] = dict(dfldict_cont_bigramas)
                    #EXEMPLO DE DICIONARIO GERADO
                    #{data1: {'bigrama1': repetições, 'bigrama2': repetições, etc...}, data2: {...}}
                    
                
            print(dict(defaultdict_datas_bigramas))  # seeeee
        
        #df_texto_data.close()     ## fecha o generator (tire para pegar todos os dados)
        
    
    #-----------------------------Parte de fora do loop-------------------------                
    
    with open('Contagem_bigramas_data.csv', 'w', encoding="utf-8") as f:
        f.write("Palavra-chave;Ocorrências\n")
        
        for key in defaultdict_datas_bigramas.keys():
            f.write("%s;%s\n"%(key,defaultdict_datas_bigramas[key]))
            
    print('A contagem de bigramas por data foi salva com sucesso no arquivo')
                          

### Função que trata e tokeniza textos
#### obs: já possui eliminação de stopwords

In [151]:
#Instância um objeto do tipo TweetTokenizer que já deixa tudo em minúsculo, trata espaços,  
# e elimina menções (mas não tira números).
tweet_tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True, preserve_case=False) 

#definição das stopwords (inclui pontuação para eliminá-la, caso apareça)
stopwords = set(stopwords.words('portuguese') + list(punctuation) + ['“','”'])

def tokenizar_texto(texto_completo):
    texto_tokenizado_lst = tweet_tokenizer.tokenize(texto_completo)
    
    if texto_tokenizado_lst[0] == "rt" and texto_tokenizado_lst[1] == ":": #Tira o "RT" caso ele esteja no inicio do tweet
        del texto_tokenizado_lst[0]
    
    palavras_sem_stopwords = [palavra for palavra in texto_tokenizado_lst if palavra not in stopwords] 
    return palavras_sem_stopwords

# EXECUÇÃO

In [152]:
##Recebe o noe do arquivo json, as colunas que se quer e o tamanho do chunk
##Retorna um generator (no formato de dataframe)
df_texto_data = json_to_dataframe('tweets_222_23abr2020.json',["text","created_at"], 30)


# Recebe uma lista de bigramas e um generator de dataframe que contenha as colunas "text" e "created_at"
contagem_bigramas_data(["distanciamento social"], df_texto_data) 

30 tweets processados...
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{'2020-04-11': {('distanciamento', 'social'): 1}}
{'2020-04-11': {('distanciamento', 'social'): 1}}
{'2020-04-11': {('distanciamento', 'social'): 1}}
{'2020-04-11': {('distanciamento', 'social'): 1}}
{'2020-04-11': {('distanciamento', 'social'): 1}}
{'2020-04-11': {('distanciamento', 'social'): 1}}
{'2020-04-11': {('distanciamento', 'social'): 1}}
{'2020-04-11': {('distanciamento', 'social'): 1}}
{'2020-04-11': {('distanciamento', 'social'): 1}}
{'2020-04-11': {('distanciamento', 'social'): 1}}
{'2020-04-11': {('distanciamento', 'social'): 1}}
{'2020-04-11': {('distanciamento', 'social'): 1}}
{'2020-04-11': {('distanciamento', 'social'): 1}}
{'2020-04-11': {('distanciamento', 'social'): 1}}
{'2020-04-11': {('distanciamento', 'social'): 1}}
{'2020-04-11': {('distanciamento', 'social'): 1}}
{'2020-04-11': {('distanciamento', 'social'): 1}}
{'2020-04-11': {('distanciamento', 'social'): 1}}
{'2020-04-11': {('distanciamento', 'social

{'2020-04-11': {('distanciamento', 'social'): 1}}
{'2020-04-11': {('distanciamento', 'social'): 1}}
{'2020-04-11': {('distanciamento', 'social'): 1}}
{'2020-04-11': {('distanciamento', 'social'): 1}}
{'2020-04-11': {('distanciamento', 'social'): 1}}
{'2020-04-11': {('distanciamento', 'social'): 1}}
{'2020-04-11': {('distanciamento', 'social'): 1}}
{'2020-04-11': {('distanciamento', 'social'): 1}}
{'2020-04-11': {('distanciamento', 'social'): 1}}
{'2020-04-11': {('distanciamento', 'social'): 1}}
{'2020-04-11': {('distanciamento', 'social'): 1}}
{'2020-04-11': {('distanciamento', 'social'): 1}}
270 tweets processados...
{'2020-04-11': {('distanciamento', 'social'): 1}}
{'2020-04-11': {('distanciamento', 'social'): 1}}
{'2020-04-11': {('distanciamento', 'social'): 1}}
{'2020-04-11': {('distanciamento', 'social'): 1}}
{'2020-04-11': {('distanciamento', 'social'): 1}}
{'2020-04-11': {('distanciamento', 'social'): 1}}
{'2020-04-11': {('distanciamento', 'social'): 1}}
{'2020-04-11': {('distan

450 tweets processados...
{'2020-04-11': {('distanciamento', 'social'): 2}}
{'2020-04-11': {('distanciamento', 'social'): 2}}
{'2020-04-11': {('distanciamento', 'social'): 2}}
{'2020-04-11': {('distanciamento', 'social'): 2}}
{'2020-04-11': {('distanciamento', 'social'): 2}}
{'2020-04-11': {('distanciamento', 'social'): 2}}
{'2020-04-11': {('distanciamento', 'social'): 2}}
{'2020-04-11': {('distanciamento', 'social'): 2}}
{'2020-04-11': {('distanciamento', 'social'): 2}}
{'2020-04-11': {('distanciamento', 'social'): 2}}
{'2020-04-11': {('distanciamento', 'social'): 2}}
{'2020-04-11': {('distanciamento', 'social'): 2}}
{'2020-04-11': {('distanciamento', 'social'): 2}}
{'2020-04-11': {('distanciamento', 'social'): 2}}
{'2020-04-11': {('distanciamento', 'social'): 2}}
{'2020-04-11': {('distanciamento', 'social'): 2}}
{'2020-04-11': {('distanciamento', 'social'): 2}}
{'2020-04-11': {('distanciamento', 'social'): 2}}
{'2020-04-11': {('distanciamento', 'social'): 2}}
{'2020-04-11': {('distan

660 tweets processados...
{'2020-04-11': {('distanciamento', 'social'): 7}}
{'2020-04-11': {('distanciamento', 'social'): 7}}
{'2020-04-11': {('distanciamento', 'social'): 7}}
{'2020-04-11': {('distanciamento', 'social'): 7}}
{'2020-04-11': {('distanciamento', 'social'): 7}}
{'2020-04-11': {('distanciamento', 'social'): 7}}
{'2020-04-11': {('distanciamento', 'social'): 7}}
{'2020-04-11': {('distanciamento', 'social'): 7}}
{'2020-04-11': {('distanciamento', 'social'): 7}}
{'2020-04-11': {('distanciamento', 'social'): 7}}
{'2020-04-11': {('distanciamento', 'social'): 7}}
{'2020-04-11': {('distanciamento', 'social'): 7}}
{'2020-04-11': {('distanciamento', 'social'): 7}}
{'2020-04-11': {('distanciamento', 'social'): 7}}
{'2020-04-11': {('distanciamento', 'social'): 7}}
{'2020-04-11': {('distanciamento', 'social'): 7}}
{'2020-04-11': {('distanciamento', 'social'): 8}}
{'2020-04-11': {('distanciamento', 'social'): 8}}
{'2020-04-11': {('distanciamento', 'social'): 8}}
{'2020-04-11': {('distan

{'2020-04-11': {('distanciamento', 'social'): 10}}
{'2020-04-11': {('distanciamento', 'social'): 10}}
{'2020-04-11': {('distanciamento', 'social'): 10}}
{'2020-04-11': {('distanciamento', 'social'): 10}}
{'2020-04-11': {('distanciamento', 'social'): 10}}
{'2020-04-11': {('distanciamento', 'social'): 10}}
{'2020-04-11': {('distanciamento', 'social'): 10}}
{'2020-04-11': {('distanciamento', 'social'): 10}}
{'2020-04-11': {('distanciamento', 'social'): 10}}
900 tweets processados...
{'2020-04-11': {('distanciamento', 'social'): 10}}
{'2020-04-11': {('distanciamento', 'social'): 10}}
{'2020-04-11': {('distanciamento', 'social'): 10}}
{'2020-04-11': {('distanciamento', 'social'): 10}}
{'2020-04-11': {('distanciamento', 'social'): 10}}
{'2020-04-11': {('distanciamento', 'social'): 10}}
{'2020-04-11': {('distanciamento', 'social'): 10}}
{'2020-04-11': {('distanciamento', 'social'): 10}}
{'2020-04-11': {('distanciamento', 'social'): 10}}
{'2020-04-11': {('distanciamento', 'social'): 10}}
{'202

...

45900 tweets processados...
{'2020-04-11': {('distanciamento', 'social'): 469}}
{'2020-04-11': {('distanciamento', 'social'): 469}}
{'2020-04-11': {('distanciamento', 'social'): 469}}
{'2020-04-11': {('distanciamento', 'social'): 469}}
{'2020-04-11': {('distanciamento', 'social'): 469}}
{'2020-04-11': {('distanciamento', 'social'): 469}}
{'2020-04-11': {('distanciamento', 'social'): 469}}
{'2020-04-11': {('distanciamento', 'social'): 469}}
{'2020-04-11': {('distanciamento', 'social'): 469}}
{'2020-04-11': {('distanciamento', 'social'): 469}}
{'2020-04-11': {('distanciamento', 'social'): 469}}
{'2020-04-11': {('distanciamento', 'social'): 469}}
{'2020-04-11': {('distanciamento', 'social'): 469}}
{'2020-04-11': {('distanciamento', 'social'): 469}}
{'2020-04-11': {('distanciamento', 'social'): 469}}
{'2020-04-11': {('distanciamento', 'social'): 469}}
{'2020-04-11': {('distanciamento', 'social'): 469}}
{'2020-04-11': {('distanciamento', 'social'): 469}}
{'2020-04-11': {('distanciamento', '

{'2020-04-11': {('distanciamento', 'social'): 471}}
{'2020-04-11': {('distanciamento', 'social'): 471}}
{'2020-04-11': {('distanciamento', 'social'): 471}}
{'2020-04-11': {('distanciamento', 'social'): 471}}
{'2020-04-11': {('distanciamento', 'social'): 471}}
{'2020-04-11': {('distanciamento', 'social'): 471}}
{'2020-04-11': {('distanciamento', 'social'): 471}}
{'2020-04-11': {('distanciamento', 'social'): 471}}
{'2020-04-11': {('distanciamento', 'social'): 471}}
{'2020-04-11': {('distanciamento', 'social'): 471}}
{'2020-04-11': {('distanciamento', 'social'): 471}}
{'2020-04-11': {('distanciamento', 'social'): 471}}
{'2020-04-11': {('distanciamento', 'social'): 471}}
{'2020-04-11': {('distanciamento', 'social'): 471}}
{'2020-04-11': {('distanciamento', 'social'): 471}}
{'2020-04-11': {('distanciamento', 'social'): 471}}
{'2020-04-11': {('distanciamento', 'social'): 471}}
{'2020-04-11': {('distanciamento', 'social'): 471}}
{'2020-04-11': {('distanciamento', 'social'): 471}}
46140 tweets

46350 tweets processados...
{'2020-04-11': {('distanciamento', 'social'): 474}}
{'2020-04-11': {('distanciamento', 'social'): 474}}
{'2020-04-11': {('distanciamento', 'social'): 474}}
{'2020-04-11': {('distanciamento', 'social'): 474}}
{'2020-04-11': {('distanciamento', 'social'): 474}}
{'2020-04-11': {('distanciamento', 'social'): 474}}
{'2020-04-11': {('distanciamento', 'social'): 474}}
{'2020-04-11': {('distanciamento', 'social'): 474}}
{'2020-04-11': {('distanciamento', 'social'): 474}}
{'2020-04-11': {('distanciamento', 'social'): 474}}
{'2020-04-11': {('distanciamento', 'social'): 474}}
{'2020-04-11': {('distanciamento', 'social'): 474}}
{'2020-04-11': {('distanciamento', 'social'): 474}}
{'2020-04-11': {('distanciamento', 'social'): 474}}
{'2020-04-11': {('distanciamento', 'social'): 474}}
{'2020-04-11': {('distanciamento', 'social'): 474}}
{'2020-04-11': {('distanciamento', 'social'): 474}}
{'2020-04-11': {('distanciamento', 'social'): 474}}
{'2020-04-11': {('distanciamento', '

{'2020-04-11': {('distanciamento', 'social'): 477}}
{'2020-04-11': {('distanciamento', 'social'): 477}}
{'2020-04-11': {('distanciamento', 'social'): 477}}
{'2020-04-11': {('distanciamento', 'social'): 477}}
{'2020-04-11': {('distanciamento', 'social'): 477}}
{'2020-04-11': {('distanciamento', 'social'): 477}}
{'2020-04-11': {('distanciamento', 'social'): 477}}
{'2020-04-11': {('distanciamento', 'social'): 477}}
{'2020-04-11': {('distanciamento', 'social'): 477}}
{'2020-04-11': {('distanciamento', 'social'): 477}}
{'2020-04-11': {('distanciamento', 'social'): 477}}
{'2020-04-11': {('distanciamento', 'social'): 477}}
{'2020-04-11': {('distanciamento', 'social'): 477}}
{'2020-04-11': {('distanciamento', 'social'): 477}}
{'2020-04-11': {('distanciamento', 'social'): 477}}
{'2020-04-11': {('distanciamento', 'social'): 477}}
{'2020-04-11': {('distanciamento', 'social'): 477}}
{'2020-04-11': {('distanciamento', 'social'): 477}}
{'2020-04-11': {('distanciamento', 'social'): 477}}
{'2020-04-11

KeyboardInterrupt: 